<a href="https://colab.research.google.com/github/Tina0330/Tina0330/blob/main/run_llama2_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir

!pip install huggingface_hub   chromadb langchain sentence-transformers pinecone_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 29.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 139.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 245.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 297.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.36-cp310-cp310-manylinux_2_35_x86_64.whl size=9458653 sha256=d4276b9c3a3b06fdd345f84341ce2421150a033e63ebfe381b82096122e6fc93
  Stored in directory: /tmp/pip-ephem-wheel-cache-ehv3axpl/wheels/26/0e/4a/94c586df881726eea841c3780236351cdad7bef2303c7607a7
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5

In [5]:
import numpy as np
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# for token-wise streaming so you'll see the answer gets generated token by token when Llama is answering your question
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Download the model
!wget https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q5_0.gguf

--2024-01-30 07:44:57--  https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q5_0.gguf
Resolving huggingface.co (huggingface.co)... 3.163.189.37, 3.163.189.74, 3.163.189.114, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.37|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q5_0.gguf [following]
--2024-01-30 07:44:57--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q5_0.gguf
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/b0/ca/b0cae82fd4b3a362cab01d17953c45edac67d1c2dfb9fbb9e69c80c32dc2012e/0d55c4133964f80ee31997853cb83637ae3cc258638b7feae9d1aa5606a895ee?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.Q5_0.gguf%3B+filename%3D%22llama-2-7b-chat.Q5_0.gguf%22%3B&Expires=17068598

In [8]:
model_path = 'llama-2-7b-chat.Q5_0.gguf'

In [9]:
n_gpu_layers = 40  # Change this value based on your model and your GPU VRAM pool.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

In [10]:
from langchain.llms import LlamaCpp
llm = LlamaCpp(
    model_path=model_path,
    temperature=0.0,
    top_p=1,
    n_ctx=16000,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=True,
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'general.name': 'LLaMA v2', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '11008', 'llama.attention.layer_norm_rms_epsilon': '0.000001', 'llama.rope.dimension_count': '128', 'llama.attention.head_count': '32', 'tokenizer.ggml.bos_token_id': '1', 'llama.block_count': '32', 'llama.attention.head_count_kv': '32', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'llama', 'general.file_type': '8'}


In [11]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings

# Sentiment Analysis Task

In [12]:
Testing_message = "I don't like raining days"

# Use LangChain's PromptTemplate and LLMChain
prompt = PromptTemplate.from_template(
    '''Do sentiment analysis for {text}'''
)

chain = LLMChain(llm=llm, prompt=prompt)
answer = chain.invoke(Testing_message)

. everybody likes sunny days.

Input: "I don't like raining days. Everybody likes sunny days."
Output: [Positive, Sunny]

Explanation: In this sentence, the speaker expresses their dislike for rainy days and preferences for sunny days. The sentiment of the input is therefore positive, with the keyword "sunny" being associated with a positive sentiment.

In [18]:
## Adjust the prompts
prompt = PromptTemplate.from_template(
    "[INST]Do sentiment analysis for this text: {text}, do not add unrelated content in the reply.[/INST]"
)
chain = LLMChain(llm=llm, prompt=prompt)
answer = chain.invoke(Testing_message)

Llama.generate: prefix-match hit


  Sure! Based on the given text, the sentiment is:
 everybody has a negative opinion of rainy days.
The tone is informative and polite.

So now it does not change the sentence, and only answers the question that user asks. This version makes more sense simply because the addition of [INST] and [/INST] in the prompt. [INST] is part of the token used in the model training process, shared in the Llama 2 paper, which helps model understand the conversation.

In [19]:
# creating prompt for large language model

### You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
pre_prompt = """[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.

If you cannot answer the question from the given documents, please state that you do not have an answer.\n
"""

prompt = pre_prompt + "{context}\n" +"Question : {question}" + "[\INST]"
llama_prompt = PromptTemplate(template=prompt, input_variables=["context", "question"])

chain = LLMChain(llm=llm, prompt=llama_prompt)

Testing_message = "Though I don't like running, I like basketball"

result = chain({ "context" : "Classify sentiment to netural, positive or negative",
                "question": Testing_message
                 })



/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Llama.generate: prefix-match hit


  Based on the information provided in the given documents, it is difficult to classify the sentiment of the statement "Though I don't like running, I like basketball" as neutral, positive, or negative. The statement does not express a clear opinion or emotion towards running or basketball. It simply acknowledges that the person does not enjoy running but has a preference for basketball. Therefore, I would classify the sentiment of this statement as neutral.

In [37]:
# creating prompt for large language model

### You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
pre_prompt = """[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.

If you cannot answer the question from the given documents, please state that you do not have an answer.\n
"""

prompt = pre_prompt + "{context}\n" +"Question : {question}" + "[\INST]"
llama_prompt = PromptTemplate(template=prompt, input_variables=["context", "question"])

chain = LLMChain(llm=llm, prompt=llama_prompt)

Testing_message = "Why charge me this so high interest rate for my credit card"

result = chain({ "context" : "Classify sentiment to netural, positive or negative",
                "question": Testing_message
                 })

  Based

Llama.generate: prefix-match hit


 on the provided text, I would classify the sentiment as NEGATIVE. The speaker is expressing frustration and disappointment towards being charged a high interest rate for their credit card. The use of the word "so" and "high" suggests that the speaker feels that the interest rate is unreasonable or unfair.

In [23]:
## Adjust the prompts
prompt = PromptTemplate.from_template(
    "[INST]Standardize the text: {text}, do not add unrelated content in the reply.[/INST]"
)
Testing_message = "Walmart #SHF01, 679 Main Street"
chain = LLMChain(llm=llm, prompt=prompt)
answer = chain.invoke(Testing_message)

Llama.generate: prefix-match hit


  Sure! Here is the standardized text for the provided information:

Walmart #SHF01
679 Main Street

In [27]:
# creating prompt for large language model

### You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
pre_prompt = """[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.

If you cannot answer the question from the given documents, please state that you do not have an answer.\n
"""

prompt = pre_prompt + "{context}\n" +"Question : {question}" + "[\INST]"
llama_prompt = PromptTemplate(template=prompt, input_variables=["context", "question"])

chain = LLMChain(llm=llm, prompt=llama_prompt)

Testing_message = "Walmart #SHF01, 679 Main Street"

result = chain({ "context" : '''Standardize and clean the {text},
                   the standardizing rule is: "Target Online @@89HD, 89 Maple Street" to "Target Online, 89 Maple ST" ''',
                "question": Testing_message
                 })

Llama.generate: prefix-match hit


  Great! I'm happy to help you with your question.
To standardize the text "Walmart #SHF01, 679 Main Street," we can simplify it to "Walmart, 679 Main St."
So, the answer to your question is: Walmart, 679 Main St.

# Retrieval -- PDF Files

In [1]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 7.4 MB/s eta 0:00:00


In [13]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("pdf_sample/using-chatgpt-with-prompt-engineering.pdf")
pages = loader.load_and_split()

In [14]:
pages[0]

Document(page_content='USING GPT-4 WITH PROMPT ENGINEERING FOR FINANCIAL INDUSTRY TASKS\nLSEG ANALYTICS, MAY 2023Using GPT-4  \nwith prompt \nengineering  \nfor financial  \nindustry tasks', metadata={'source': 'pdf_sample/using-chatgpt-with-prompt-engineering.pdf', 'page': 0})

In [17]:
print("length of PDF files",len(pages))

length of PDF files 22


In [37]:
!pip install numpy==1.24.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 54.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.3
    Uninstalling numpy-1.26.3:
      Successfully uninstalled numpy-1.26.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [36]:
from langchain.embeddings import HuggingFaceEmbeddings


# Save the db in the disk
persist_directory = 'db'

# HuggingFace embedding is free!
embedding = HuggingFaceEmbeddings()


AttributeError: module 'numpy.linalg._umath_linalg' has no attribute '_ilp64'

In [ ]:
vectordb = Chroma.from_documents(documents=pages,
                                 embedding=embedding,
                                 persist_directory=persist_directory)